In [1]:
library(tidyverse)
library(lme4)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:tidyr’:

    expand



In [2]:
aggregated_paris_listings_df = data.frame()

for (csv_file in list.files('paris-listings')) {
    
    listing_df = read_csv(paste0('~/ml-projects/regression-tutorial/paris-listings/',csv_file))
    
    aggregated_paris_listings_df = bind_rows(aggregated_paris_listings_df, listing_df)

}

Parsed with column specification:
cols(
  room_id = col_double(),
  host_id = col_double(),
  room_type = col_character(),
  borough = col_logical(),
  neighborhood = col_character(),
  reviews = col_double(),
  overall_satisfaction = col_double(),
  accommodates = col_double(),
  bedrooms = col_double(),
  price = col_double(),
  minstay = col_double(),
  latitude = col_double(),
  longitude = col_double(),
  last_modified = col_datetime(format = "")
)
Parsed with column specification:
cols(
  room_id = col_double(),
  host_id = col_double(),
  room_type = col_character(),
  borough = col_logical(),
  neighborhood = col_character(),
  reviews = col_double(),
  overall_satisfaction = col_double(),
  accommodates = col_double(),
  bedrooms = col_double(),
  price = col_double(),
  minstay = col_double(),
  latitude = col_double(),
  longitude = col_double(),
  last_modified = col_datetime(format = "")
)
Parsed with column specification:
cols(
  room_id = col_double(),
  host_id = col_do

In [3]:
aggregated_paris_listings_df %>%
    mutate(room_id = factor(room_id),
           host_id = factor(host_id)) %>%
    select(-country, -survey_id, -bathrooms, -borough,
           -minstay, -location, -last_modified) ->
    aggregated_paris_listings_df

In [4]:
aggregated_paris_listings_df %>%
    mutate(overall_satisfaction = replace(overall_satisfaction, overall_satisfaction == 0, NA)) ->
    aggregated_paris_listings_df

In [5]:
names(aggregated_paris_listings_df)

[1] "room_id"              "host_id"              "room_type"           
 [4] "neighborhood"         "reviews"              "overall_satisfaction"
 [7] "accommodates"         "bedrooms"             "price"               
[10] "latitude"             "longitude"            "city"                
[13] "name"                 "property_type"

In [7]:
lmer(price ~ 1 + (1 | neighborhood), 
     data = aggregated_paris_listings_df,
     REML = FALSE) %>%

summary()

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: price ~ 1 + (1 | neighborhood)
   Data: aggregated_paris_listings_df

     AIC      BIC   logLik deviance df.resid 
10342010 10342045 -5171002 10342004   837817 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-2.052 -0.386 -0.166  0.136 96.884 

Random effects:
 Groups       Name        Variance Std.Dev.
 neighborhood (Intercept)  1696     41.18  
 Residual                 13432    115.90  
Number of obs: 837820, groups:  neighborhood, 80

Fixed effects:
            Estimate Std. Error t value
(Intercept)  128.418      4.607   27.87

In [9]:
cat("Amount of variance explained by variation in neighborhoods alone", 1696/(1696+13432))

Amount of variance explained by variation in neighborhoods alone 0.11211

In [10]:
lmer(price ~ overall_satisfaction + accommodates + reviews + (1 | neighborhood),
     data = na.omit(aggregated_paris_listings_df),
     REML = FALSE) %>%

summary()

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: price ~ overall_satisfaction + accommodates + reviews + (1 |  
    neighborhood)
   Data: na.omit(aggregated_paris_listings_df)

      AIC       BIC    logLik  deviance  df.resid 
 923587.2  923642.7 -461787.6  923575.2     76597 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
 -5.323  -0.306  -0.048   0.188 110.348 

Random effects:
 Groups       Name        Variance Std.Dev.
 neighborhood (Intercept)   928     30.46  
 Residual                 10039    100.20  
Number of obs: 76603, groups:  neighborhood, 80

Fixed effects:
                       Estimate Std. Error t value
(Intercept)          -110.24665    5.81191 -18.969
overall_satisfaction   26.65025    0.99363  26.821
accommodates           33.33577    0.23899 139.486
reviews                -0.09939    0.01006  -9.883

Correlation of Fixed Effects:
            (Intr) ovrll_ accmmd
ovrll_stsfc -0.795              
accommodats -0.130  0.002       
re

In [11]:
nrow(na.omit(aggregated_paris_listings_df))

[1] 76603

In [12]:
head(InstEval)

s,d,studage,lectage,service,dept,y
<fct>,<fct>,<ord>,<ord>,<fct>,<fct>,<int>
1,1002,2,2,0,2,5
1,1050,2,1,1,6,2
1,1582,2,2,0,2,5
1,2050,2,2,1,3,3
2,115,2,1,0,5,2
2,756,2,1,0,5,4


In [13]:
lmer(y ~ service + lectage + studage + (1|d) + (1|s),
     data = InstEval) %>%

summary()

Linear mixed model fit by REML ['lmerMod']
Formula: y ~ service + lectage + studage + (1 | d) + (1 | s)
   Data: InstEval

REML criterion at convergence: 237629.4

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-3.08684 -0.74626  0.04111  0.77158  3.15418 

Random effects:
 Groups   Name        Variance Std.Dev.
 s        (Intercept) 0.1064   0.3262  
 d        (Intercept) 0.2670   0.5167  
 Residual             1.3836   1.1762  
Number of obs: 73421, groups:  s, 2972; d, 1128

Fixed effects:
             Estimate Std. Error t value
(Intercept)  3.224843   0.019428 165.986
service1    -0.070844   0.013367  -5.300
lectage.L   -0.186006   0.016095 -11.557
lectage.Q    0.023770   0.012426   1.913
lectage.C   -0.024122   0.013051  -1.848
lectage^4   -0.020408   0.013468  -1.515
lectage^5   -0.038681   0.015122  -2.558
studage.L    0.095621   0.018920   5.054
studage.Q    0.006022   0.016233   0.371
studage.C    0.016799   0.016028   1.048

Correlation of Fixed Effects:
  